In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter
import seaborn as sns
import pandas as pd
import datetime


anomaly = "qtran__2022-09-09_11-22-30_668"

indi_pos = "qtran__2022-09-09_11-49-19_1"

obvious = "qtran__2022-09-09_12-09-12_908"

normal = "qtran__2022-09-09_19-52-13_391"

indi_pos1 = "qtran__2022-09-10_11-22-09_553"

test = "qtran__2022-09-15_15-57-15_973"

dir_name = "AA-2800"


env_name = "3s_vs_3z"

save_path = os.path.join(os.getcwd(), "results", "models", "custom", env_name, dir_name)

with open(os.path.join(save_path, "pro_pos.npy"), "rb") as pro_pos_f:
    fsz = os.fstat(pro_pos_f.fileno()).st_size
    pro_pos = np.load(pro_pos_f)
    while pro_pos_f.tell() < fsz:
        pro_pos = np.vstack((pro_pos, np.load(pro_pos_f)))
    pro_x = pro_pos[:, 0]
    pro_y = pro_pos[:, 1]
    pro_step = pro_pos[:, 2]

with open(os.path.join(save_path, "ant_pos.npy"), "rb") as ant_pos_f:
    fsz = os.fstat(ant_pos_f.fileno()).st_size
    ant_pos = np.load(ant_pos_f)
    while ant_pos_f.tell() < fsz:
        ant_pos = np.vstack((ant_pos, np.load(ant_pos_f)))
    ant_x = ant_pos[:, 0]
    ant_y = ant_pos[:, 1]
    ant_step = ant_pos[:, 2]

def plot_coords(x, y, s):
    fig, ax = plt.subplots()
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=500, density=True)
    heatmap = gaussian_filter(heatmap, sigma=s)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    ax.imshow(heatmap.T, extent=extent, origin='lower', cmap=cm.jet)
    plt.gca().set_aspect('equal', adjustable='box')
    return plt, fig

def select_first(x, y, steps, max):
    x = np.array([x[x_i] for x_i in range(len(x)) if steps[x_i] < max])
    y = np.array([y[y_i] for y_i in range(len(y)) if steps[y_i] < max])
    steps = np.array([steps[steps_i] for steps_i in range(len(steps)) if steps[steps_i] < max])
    return x, y, steps

def add_corners(x, y, steps):
    x = list(x)
    y = list(y)
    steps = list(steps)
    x.append(-0.5)
    x.append(0.5)
    y.append(-0.5)
    y.append(0.5)
    steps.append(0)
    steps.append(0)
    x = np.array(x)
    y = np.array(y)
    steps = np.array(steps)
    return x, y, steps

def plot_sns_hist(x, y, steps, s):
    df = pd.DataFrame({"x": x, "y": y, "step": steps})

    color = sns.color_palette("coolwarm", as_cmap=True)

    norm = plt.Normalize(df['step'].min(), df['step'].max())
    sm = plt.cm.ScalarMappable(cmap = color, norm=norm)
    sm.set_array([])

    hist = sns.histplot(df, x="x", y="y", hue="step", bins=s, pthresh=.0002, palette=color)
    hist.get_legend().remove()
    hist.figure.colorbar(sm)

    hist.patch.set_alpha(0)
    fig = hist.get_figure()
    return fig

def save_plot(label, fig):
    timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    plot_dir = os.path.join(os.getcwd(), 'plots')
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
    fig.savefig(os.path.join(plot_dir, '{}_pos_{}.png'.format(label, timestamp)), dpi=300, bbox_inches="tight")
    fig.clf()

s = 16
max_step = 75


ant_x, ant_y, ant_step = add_corners(ant_x, ant_y, ant_step)
pro_x, pro_y, pro_step = add_corners(pro_x, pro_y, pro_step)

ant_x, ant_y, ant_step = select_first(ant_x, ant_y, ant_step, max_step)
pro_x, pro_y, pro_step = select_first(pro_x, pro_y, pro_step, max_step)

# ant_plt, ant_fig = plot_coords(ant_x, ant_y, s)
# ant_fig.savefig(os.path.join(save_path, "antagonist_pos.png"), dpi=300, bbox_inches="tight")
# ant_plt.show()
# ant_fig.clf()
#
# pro_plt, pro_fig = plot_coords(pro_x, pro_y, s)
# pro_fig.savefig(os.path.join(save_path, "protagonist_pos.png"), dpi=300, bbox_inches="tight")
# pro_plt.show()
# pro_fig.clf()

label_pro = "protagonist"
label_ant = "antagonist"
label_diff = "difference"

fig = plot_sns_hist(ant_x, ant_y, ant_step, s)
save_plot(label_ant, fig)
fig = plot_sns_hist(pro_x, pro_y, pro_step, s)
save_plot(label_pro, fig)


# x = pro_x
# y = pro_y
#
# color = sns.color_palette("coolwarm", as_cmap=True)
# H, yedges, xedges = np.histogram2d(y, x, bins=20)
#
#
# plt1 = plt.pcolormesh(xedges, yedges, H, cmap=color, rasterized=True, shading="flat")
#
# plt.gca().set_aspect('equal', adjustable='box')
# plt.show()